In [ ]:
#pip install pandas openpyxl google-generativeai

In [ ]:
import os
import pandas as pd
import google.generativeai as genai
from IPython.display import display, Markdown

# --- ฟังก์ชันสำหรับอ่าน API Key จากไฟล์ ---
def get_api_key_from_file(filepath="GOOGLE_API_KEY.txt"):
    """อ่าน API Key จากไฟล์ที่ระบุ"""
    if not os.path.exists(filepath):
        print(f"Error: API Key file not found at '{filepath}'")
        print("Please create the file and place your API key inside.")
        return None
    try:
        with open(filepath, "r") as f:
            api_key = f.read().strip()
        if not api_key:
            print(f"Error: The file '{filepath}' is empty.")
            return None
        return api_key
    except Exception as e:
        print(f"Error reading API Key from file: {e}")
        return None

# --- การตั้งค่า API Key ---
api_key = get_api_key_from_file()

if api_key:
    genai.configure(api_key=api_key)
    print("API Key loaded successfully from GOOGLE_API_KEY.txt")
else:
    print("API Key not loaded. Please check the file and try again.")

def find_trending_file(directory="."):
    """ค้นหาไฟล์ trending ที่มีวันที่ในชื่อไฟล์"""
    for filename in os.listdir(directory):
        if filename.startswith("trending_TH_1d_") and filename.endswith(".csv"):
            return filename
    return None

def read_file_content(filename):
    """อ่านเนื้อหาจากไฟล์ .xlsx หรือ .csv แล้วแปลงเป็น string"""
    try:
        if filename.endswith('.xlsx'):
            df = pd.read_excel(filename)
        elif filename.endswith('.csv'):
            df = pd.read_csv(filename)
        else:
            return None # ไม่ใช่ไฟล์ที่รองรับ
        
        # แปลง DataFrame เป็น string ในรูปแบบ CSV เพื่อให้โมเดลเข้าใจง่าย
        return df.to_csv(index=False)
    except Exception as e:
        return f"Error reading {filename}: {e}"

def analyze_restaurant_data_and_save():
    """
    รวบรวมข้อมูล, วิเคราะห์ด้วย Gemini API, และบันทึกผลลัพธ์ลงไฟล์
    """
    if not api_key:
        print("\nCannot proceed without a valid API Key.")
        return

    # --- 1. กำหนดชื่อไฟล์และ Prompt ---
    trending_file = find_trending_file()
    if not trending_file:
        print("Error: Could not find the trending CSV file in the current directory.")
        return

    filenames = [
        "sales_insight.xlsx",
        "gpos_insight.xlsx",
        "ggmap_review.xlsx",
        trending_file,
        "nd_plan.xlsx",
        "Market_Basket_rules.xlsx"
    ]

    user_prompt = """นายคือนักการตลาดมืออาชีพสำหรับร้านอาหาร กุ้งเตี๋ยวแคะ โดยจะวิเคราะห์ตัดสินใจจาก    

   1. Insight ที่ได้รับมาจาก "sales_insight.xlsx" + "gpos_insight.xlsx" + "ggmap_review.xlsx"   + "Market_Basket_rules.xlsx"

   2. กระแสประจำวันจาก "trending_TH_1d_yyyymmdd.csv"   

   3. แผนแนวทางการพัฒนาของธุรกิจ จาก "nd_plan.xlsx"   

   ตอบเป็น   

   1. วันที่ปัจจุบัน : yyyy-mm-dd   

   1. จุดแข็ง :   

   2. Pain point (เรียงจากสำคัญมากไปน้อย) :    

   3. Actionable Insight (เรียงจากส่งผลต่อธุรกิจมากไปน้อย) :    

   4. แนวทางการทำการตลาด พร้อมเหตุผลจาก Actionable insight :    

   5. หัวข้อการทำคอนเท้นประจำวันตามกระแสและสภานการณ์ธุรกิจจาก Actionable insight (7ตัวเลือก ตามกระแสที่เกี่ยวข้อง):    

   6. แนวทางการจับคู่เมนู จาก insight market basket analysis  พร้อมเหตุผล ("Market_Basket_rules.xlsx" + Insight อื่นๆ) :    

   7. แนวทางการทำโปรโมชั่นระยะสั้นเพื่อความได้เปรียบคู่แข่งสำหรับลูกค้าหน้าร้าน (อิงจาก googlemap_review_yyyy-mm-dd เป็นหลักประกอบกับ insight) :   

   8. แนวทางการทำโปรโมชั่นระยะสั้นเพื่อความได้เปรียบคู่แข่งสำหรับลูกค้าในแอพ grab (อิงจาก grab_review_yyyy-mm-dd และ shopee_review_yyyy-mm-dd เป็นหลักประกอบกับ insight จากdelivery_sales_insight_yyyy-mm-dd และ insight อื่นรองลงมา) :  

  9. ช่องว่างในการพัฒนา (เรียงจากการพัฒนาที่สำคัญมากไปน้อย) :
"""

    # --- 2. อ่านข้อมูลจากไฟล์และสร้าง Prompt ที่สมบูรณ์ ---
    full_prompt = [user_prompt]
    print("Reading files...")
    
    for fname in filenames:
        if not os.path.exists(fname):
            print(f"Warning: File '{fname}' not found. Skipping.")
            continue
        
        print(f"- Reading {fname}...")
        content = read_file_content(fname)
        
        full_prompt.append(f"\n--- START OF FILE: {fname} ---\n")
        full_prompt.append(content)
        full_prompt.append(f"\n--- END OF FILE: {fname} ---\n")
    
    final_prompt_string = "".join(full_prompt)
    
    # --- 3. เรียกใช้ Gemini API ---
    print("\nSending data to Gemini API for analysis... (This may take a moment)")
    
    try:
        model = genai.GenerativeModel('gemini-1.5-pro-latest')
        response = model.generate_content(final_prompt_string)
        
        # --- 4. แสดงผลลัพธ์และบันทึกลงไฟล์ ---
        print("\n--- Analysis Report from Gemini ---")
        display(Markdown(response.text)) # แสดงผลใน Notebook
        
        # บันทึกผลลัพธ์ลงไฟล์ AI_result.txt
        output_filename = "AI_result.txt"
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(response.text)
        print(f"\nAnalysis successfully saved to '{output_filename}'")

    except Exception as e:
        print(f"\nAn error occurred while calling the Gemini API: {e}")

# ==============================================================================
# Cell 3: Execute the analysis
# ==============================================================================
analyze_restaurant_data_and_save()